# 🚗 AV Simulation Runner - Google Colab

This notebook demonstrates how to run the Autonomous Vehicle Simulation on Google Colab with data collection and visualization.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aanshshah/av-simulation/blob/main/examples/notebooks/02_simulation_runner.ipynb)

## Setup Environment

In [ ]:
# Setup virtual display
from pyvirtualdisplay import Display
import os

display = Display(visible=0, size=(1200, 800))
display.start()
os.environ['DISPLAY'] = ':' + str(display.display)

print('✅ Virtual display ready')

## Run Simulation

In [ ]:
# Import simulation modules
from av_simulation.core.simulation import Simulation
from av_simulation.data.repository import DataRepository

# Create and run simulation
sim = Simulation(enable_data_collection=True)
data = sim.run_simulation(duration=60)

print('🚗 Simulation completed!')

## View Results

In [ ]:
# Display simulation results
import matplotlib.pyplot as plt

# Plot some basic metrics
if data:
    plt.figure(figsize=(10, 6))
    plt.plot(data.timestamps, data.speeds)
    plt.title('Vehicle Speed Over Time')
    plt.xlabel('Time (s)')
    plt.ylabel('Speed (m/s)')
    plt.show()
else:
    print('No data collected')